In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
import torchmetrics
import numpy as np

In [2]:
import wandb
import os
from dotenv import load_dotenv

load_dotenv()
!wandb login {os.getenv("WANDB_API_KEY")}

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nikol/.netrc


In [2]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
import engine
import experiments_maker


!wandb login os.environ['WANDB_API_KEY']

modes = ["disabled", "online"]
device = "cuda" if torch.cuda.is_available() else "cpu"

epochs = 50
lr = 1e-3
classes = 10
loss_fn = nn.CrossEntropyLoss()
accuracy_fn = torchmetrics.Accuracy(task='multiclass', num_classes=classes, average='macro')

batch_size=16

# datasets_names = ['mnist', 'tmnist','fashion_mnist', 'cifar10']
# optimizers_names = ['SGD', 'HessianFree', 'PB_BFGS', 'K_BFGS', 'K_LBFGS']
# models_names = ['SmallCNN', 'DepthCNN', 'WidthCNN', 'DepthWidthCNN']

# experimental run v0.1
datasets_names = ['tmnist']
optimizers_names = ['SGD']#, 'HessianFree']
models_names = ['SmallCNN']

for dataset_name in datasets_names:
    for optimizer_name in optimizers_names:
        for model_name in models_names:
            config = dict(
                epochs=epochs,
                classes=classes,
                learning_rate=lr,
                batch_size=batch_size,
                dataset=dataset_name, # iterable
                optimizer=optimizer_name, # iterable
                model=model_name, # iterable
                architecture="CNN"
            )

            with wandb.init(project="baselines_cnn", config=config, mode=modes[1]):
                config = wandb.config
                # make the model, data and optimization
                model, train_dataloader, test_dataloader, optimizer = experiments_maker.make(config, device)
                engine.train(config['epochs'], model, train_dataloader, test_dataloader, loss_fn, optimizer, accuracy_fn, device)

            wandb.finish()

  0%|          | 0/50 [00:00<?, ?it/s]